In [1]:
import json
import csv
import random
import time
import torch
import torch.nn as nn
import numpy as np

In [108]:
class Baseline(nn.Module):
    def __init__(self, in_size = 512, out_size = 7):
        super(Baseline, self).__init__()

        self.W = nn.Linear(in_size, 7)
        self.out = nn.LogSoftmax(2)
        
    def forward(self, x):
        x = self.W(x)
        return self.out(x)
net = torch.load('models/USE-linear')
net.eval()

Baseline(
  (W): Linear(in_features=512, out_features=7, bias=True)
  (out): LogSoftmax()
)

In [115]:
use_en_vecs_t = np.load('data/snips_processed/USE-en.npy')
w2v_sv_vecs_t = np.load('data/snips_processed/W2V-sv.npy')
elmo_sv_vecs_t = np.load('data/snips_processed/ELMO-sv.npy')

labs = np.load('data/snips_processed/labs.npy')

w2v_sv_vecs = np.squeeze(w2v_sv_vecs_t, axis=1)
use_en_vecs = np.squeeze(use_en_vecs_t, axis=1)
elmo_sv_vecs = np.squeeze(elmo_sv_vecs_t, axis=1)

#appendd bias
w2v_sv_vecs = np.hstack([w2v_sv_vecs, np.array([np.ones(len(w2v_sv_vecs))]).T])
elmo_sv_vecs = np.hstack([elmo_sv_vecs, np.array([np.ones(len(elmo_sv_vecs))]).T])


en_train = use_en_vecs[: 5000]
en_val = use_en_vecs[5000:10000]
en_test = use_en_vecs[10000:]

sv_train = w2v_sv_vecs[: 5000]
sv_val = w2v_sv_vecs[5000:10000]
sv_test = w2v_sv_vecs[10000:]

sv_train_e = elmo_sv_vecs[: 5000]
sv_val_e = elmo_sv_vecs[5000:10000]
sv_test_e = elmo_sv_vecs[10000:]

labs_train = labs[: 5000]
labs_val = labs[5000:10000]
labs_test = labs[10000:]
use_en_vecs.shape, w2v_sv_vecs.shape, elmo_sv_vecs.shape, len(labs)

((13784, 512), (13784, 301), (13784, 1025), 13784)

In [103]:
np.linalg.matrix_rank(w2v_sv_vecs), len(list(np.sum(w2v_sv_vecs, axis = 1))),len(set(np.sum(w2v_sv_vecs, axis = 1)))

(84, 13784, 13500)

In [117]:
# a x = b

x, res, rank, sing = np.linalg.lstsq(sv_train, en_train)


x_e, res_elmo, rank_elmo, sing_elmo = np.linalg.lstsq(sv_train_e, en_train)


/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/clarity/Code/virtualenvs/main/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  


In [137]:
print('Transform matrix for w2v {}, \nTransform m for elmo {}'.format(x.shape, x_e.shape))
print('train error of regression for w2w {:.3}, elmo {:.3}, \ntest error w2v {:.3} elmo {:.3}'.format(
    np.linalg.norm(en_train -(sv_train@x)),\
    np.linalg.norm(en_train -(sv_train_e@x_e)), \
    np.linalg.norm(en_val -(sv_val@x)),\
    np.linalg.norm(en_val -(sv_val_e@x_e)))) 


Transform matrix for w2v (301, 512), 
Transform m for elmo (1025, 512)
train error of regression for w2w 54.7, elmo 40.7, 
test error w2v 3.59e+10 elmo 57.0


In [134]:
def eval(model, vectors, labels):
    with torch.no_grad():
        vectors = list(map(lambda x: np.array([x]) , vectors))
        #labels = list(map(lambda x: np.array([x]) , labels))
        vectors = torch.tensor(list(vectors)).float()
        labels = torch.tensor(labels)
            
        model_out = model.forward(vectors)
        right = 0
        
        for i  in range(len(model_out)):
            k, v = model_out[i].topk(1)
            predicted, true = v.item(), labels[i].item()
            if predicted == true:
                right +=1
        return right/len(model_out)
    
print('USE vectors on USE model testset: {:.3}'.format(eval(net, en_test, labs_test)))
print('Transformed w2v vecs from val and test {:.3}, {:.3}'.format(eval(net, sv_val@x, labs_val),
                                                             eval(net, sv_test@x, labs_test)))
print('Transformed elmo vecs from val and test {:.3}, {:.3}'.format(eval(net, sv_val_e@x_e, labs_val),
                                                             eval(net, sv_test_e@x_e, labs_test)))


USE vectors on USE model testset: 0.953
Transformed w2v vecs from val and test 0.707, 0.72
Transformed elmo vecs from val and test 0.88, 0.878
